<a href="https://colab.research.google.com/github/DHIVYASRI-D/Comparing-Transformer-Models-for-Token-Based-Code-Completion-in-Python/blob/main/3_codet5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP & INSTALL


In [ ]:
!pip install transformers datasets evaluate accelerate

# IMPORT LIBRARIES

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import math

# LOAD DATA

In [ ]:
!pip install -U datasets

In [ ]:
dataset = load_dataset("code_search_net", "python")
train_data = dataset["train"].select(range(1000))
val_data = dataset["validation"].select(range(200))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

# Load tokenizer and mode

In [ ]:
model_checkpoint = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

# Preprocess function (token-level shifting)

In [ ]:
def preprocess(example):
    code = example["func_code_string"]
    tokens = tokenizer(code, truncation=True, padding="max_length", max_length=128)

    input_ids = tokens["input_ids"]
    labels = input_ids[1:] + [tokenizer.pad_token_id]  # Shift left
    inputs = input_ids[:-1] + [tokenizer.pad_token_id]

    # Mask pad tokens
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

    tokens["input_ids"] = inputs
    tokens["labels"] = labels
    return tokens

# Tokenize datasets

In [ ]:
tokenized_train = train_data.map(preprocess, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess, remove_columns=val_data.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

# Data collator for padding

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training setup

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./codet5-base-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    predict_with_generate=False,
    report_to="none"
)

TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

# Trainer

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-10-b3ba996c13b5>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Train

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.010400,0.001328


TrainOutput(global_step=500, training_loss=0.09893029451370239, metrics={'train_runtime': 4712.7229, 'train_samples_per_second': 0.212, 'train_steps_per_second': 0.106, 'total_flos': 152239472640000.0, 'train_loss': 0.09893029451370239, 'epoch': 1.0})

# Evaluation function

In [ ]:
def evaluate_codeT5(model, tokenizer, eval_dataset):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    for i in range(len(eval_dataset)):
        input_ids = torch.tensor(eval_dataset[i]["input_ids"]).unsqueeze(0)
        labels = torch.tensor(eval_dataset[i]["labels"]).unsqueeze(0)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
            logits = outputs.logits
            loss = outputs.loss

        predictions = torch.argmax(logits, dim=-1)
        mask = labels != -100

        correct += ((predictions == labels) & mask).sum().item()
        total += mask.sum().item()
        total_loss += loss.item()

    accuracy = correct / total if total > 0 else 0.0
    perplexity = math.exp(total_loss / len(eval_dataset))
    return accuracy, perplexity

# Evaluate

In [ ]:
accuracy, perplexity = evaluate_codeT5(model, tokenizer, tokenized_val)
print(f"Accuracy: {accuracy:.4f}")
print(f"Perplexity: {perplexity:.4f}")

Accuracy: 0.9997
Perplexity: 1.0015


In [ ]:
# Choose a directory to save
save_dir = "./codeT5-base-finetuned"

# Save model
model.save_pretrained(save_dir)

# Save tokenizer
tokenizer.save_pretrained(save_dir)

print(f"Model and tokenizer saved to {save_dir}")


Model and tokenizer saved to ./codeT5-base-finetuned


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
save_dir = "/content/drive/MyDrive/fine-tuned-models/codeT5-base-finetuned"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Model and tokenizer saved permanently to {save_dir}")


Model and tokenizer saved permanently to /content/drive/MyDrive/fine-tuned-models/codeT5-base-finetuned


# UI

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

# Load fine-tuned CodeT5 model
model_dir = "/content/drive/MyDrive/fine-tuned-models/codeT5-base-finetuned"
model = T5ForConditionalGeneration.from_pretrained(model_dir).to("cpu").eval()
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Prediction function
def predict_codet5(input_code, max_tokens):
    prompt = f"# Python 3\n# Complete the following function:\n{input_code.strip()}\n"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=max_tokens,
    num_beams=5,
    early_stopping=True
    )
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)



# Gradio UI with a slider
with gr.Blocks() as demo:
    gr.Markdown("## Code Completion with Fine-Tuned CodeT5")
    code_input = gr.Textbox(label="Enter partial Python function", lines=5, placeholder="e.g., def add(a, b):")
    token_slider = gr.Slider(minimum=32, maximum=128, value=64, step=1, label="Max Tokens to Generate")
    output = gr.Textbox(label="Predicted Completion")

    btn = gr.Button("Generate")
    btn.click(fn=predict_codet5, inputs=[code_input, token_slider], outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7eaa4d73b04ac45782.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# checking

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

# Load the raw CodeT5-base model and tokenizer from Hugging Face
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base").to("cpu").eval()
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")


In [ ]:
def predict_codet5(input_code, max_tokens):
    prompt = f"code completion: {input_code.strip()}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_tokens,
        do_sample=True,           # Sampling ON
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## Code Completion with Raw CodeT5-Base")
    code_input = gr.Textbox(label="Enter partial Python code", lines=5)
    token_slider = gr.Slider(minimum=16, maximum=128, value=64, step=1, label="Max Tokens")
    output = gr.Textbox(label="Predicted Completion")

    btn = gr.Button("Generate")
    btn.click(fn=predict_codet5, inputs=[code_input, token_slider], outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef37c671a4be57fb48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
